In [19]:
# Inhaltsbasiertes Empfehlungssystem für Netflix-Filme

**Gruppe 1:** <br>
Theen, Johannes (TH München)<br>
Utz, Elisabeth (OTH Amberg-Weiden)<br>
Yaruchyk, Oleg (TH München)

In [20]:
import pandas as pd
import numpy as np


In [21]:
raw_data = pd.read_csv('app/static/datasets/netflix_titles.csv') #read data
raw_data = raw_data.fillna('') #delete all NAN
print(len(raw_data))

6234


 ## Input Data

In [22]:
raw_data.head()



,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [23]:
raw_data['type']= raw_data['type'].str.replace(' ', '')
raw_data['description'] = raw_data['description'].str.lower() #Set description value

def organize_data(data): #Save name and surname as one word
    data = data.str.replace(', ',',')
    data = data.str.replace(' ','')
    data = data.str.lower()
    data = data.str.replace(',',', ')

    return data

raw_data['director'] = organize_data(raw_data['director'])
raw_data['listed_in'] = organize_data(raw_data['listed_in'])
raw_data['cast'] = organize_data(raw_data['cast'])
raw_data['country'] = organize_data(raw_data['country'])
raw_data['type'] = organize_data(raw_data['type'])

In [24]:
raw_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,movie,Norm of the North: King Sized Adventure,"richardfinn, timmaltby","alanmarriott, andrewtoth, briandobson, colehow...","unitedstates, india, southkorea, china","September 9, 2019",2019,TV-PG,90 min,"children&familymovies, comedies",before planning an awesome wedding for his gra...
1,80117401,movie,Jandino: Whatever it Takes,,jandinoasporaat,unitedkingdom,"September 9, 2016",2016,TV-MA,94 min,stand-upcomedy,jandino asporaat riffs on the challenges of ra...
2,70234439,tvshow,Transformers Prime,,"petercullen, sumaleemontano, frankwelker, jeff...",unitedstates,"September 8, 2018",2013,TV-Y7-FV,1 Season,kids'tv,"with the help of three human allies, the autob..."
3,80058654,tvshow,Transformers: Robots in Disguise,,"willfriedle, darrencriss, constancezimmer, kha...",unitedstates,"September 8, 2018",2016,TV-Y7,1 Season,kids'tv,when a prison ship crash unleashes hundreds of...
4,80125979,movie,#realityhigh,fernandolebrija,"nestacooper, katewalsh, johnmichaelhiggins, ke...",unitedstates,"September 8, 2017",2017,TV-14,99 min,comedies,when nerdy high schooler dani finally attracts...


In [25]:
import string
# Delete all punctuation
raw_data['cast'] = [row.translate(str.maketrans("","", string.punctuation)) for row in raw_data['cast']]
raw_data['listed_in'] = [row.translate(str.maketrans("","", string.punctuation)) for row in raw_data['listed_in']]
raw_data['director'] = [row.translate(str.maketrans("","", string.punctuation)) for row in raw_data['director']]
raw_data['country'] = [row.translate(str.maketrans("","", string.punctuation)) for row in raw_data['country']]
raw_data['rating'] = [row.translate(str.maketrans("","", string.punctuation)) for row in raw_data['rating']]

In [26]:
# create net BoW columns (genre and team)
raw_data['ganre']  = raw_data['type'] +' ' +  raw_data['listed_in']+ ' ' + raw_data['rating']
raw_data['team']  = raw_data['director'] + ' ' + raw_data['cast']
final_data = raw_data[['show_id','title','description', 'team', 'ganre','country']]
final_data['BoW_all'] = final_data['country']+' '+final_data['description']+' '+final_data['team']+' '+final_data['ganre']



/home/sotpak/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [27]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer()
countvector= CountVectorizer()
description_matrix = tfidf_vectorizer.fit_transform(raw_data['description'])
ganre_matrix = countvector.fit_transform(final_data['team'])
team_matrix = countvector.fit_transform(final_data['ganre'])
country_matrix = countvector.fit_transform(final_data['country'])

In [28]:
all_matrix = countvector.fit_transform(final_data['BoW_all'])
indices = pd.Series(final_data['title'])

In [29]:
# generating the cosine similarity matrix
from sklearn.metrics.pairwise import cosine_similarity
description_sim = cosine_similarity(description_matrix, description_matrix)
ganre_sim = cosine_similarity(ganre_matrix, ganre_matrix)
team_sim = cosine_similarity(team_matrix, team_matrix)
country_sim = cosine_similarity(country_matrix,country_matrix)

all_sim = cosine_similarity(all_matrix,all_matrix)

In [30]:
cosine_sim  = 2*ganre_sim + 2*team_sim + description_sim + country_sim
# we can configurate cooef to similarity table (in this case all coef for team and ganre will be 2x)

In [31]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit_transform(cosine_sim)
scaled_cosim = scaler.transform(cosine_sim) #just scale the matrix in range 0-1

cosine_sim = scaled_cosim

In [32]:
cosine_distance = np.sqrt(1 - scaled_cosim) #calculate the distance matrix
cosine_distance_pre = cosine_distance + cosine_distance.T - np.diag(np.diag(cosine_distance)) #same distance matrix but symmetric

In [33]:
def recommendations(title):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_distance[idx]).sort_values(ascending = True)

    #print(score_series)
    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(raw_data['title'])[i])
        
    return recommended_movies

recommendations('Spider-Man 3')

['John Carter',
 'Scorpion King 5: Book of Souls',
 'Black Panther',
 'Dragonheart 3: The Sorcerer',
 'Dragonheart: Battle for the Heartfire',
 'Hulk Vs.',
 'Avengers: Infinity War',
 'Skyline',
 'Superman Returns',
 'Æon Flux']

In [34]:

#######################

# quick find function
filt = raw_data['title'].str.find('Bill')>=0
raw_data.loc[filt]


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,ganre,team
160,80184771,tvshow,Inside Bill's Brain: Decoding Bill Gates,,billgates,unitedstates,"September 20, 2019",2019,TV14,1 Season,docuseries sciencenaturetv,take a trip inside the mind of bill gates as t...,tvshow docuseries sciencenaturetv TV14,billgates
293,81060174,movie,Bill Burr: Paper Tiger,mikebinder,billburr,unitedstates unitedkingdom,"September 10, 2019",2019,TVMA,67 min,standupcomedy,"bill burr unloads on outrage culture, male fem...",movie standupcomedy TVMA,mikebinder billburr
1335,70174733,movie,Bill Burr: Let It Go,shannonhartman,billburr,unitedstates,"November 10, 2017",2010,NR,65 min,standupcomedy,the musings of comedian bill burr are let loos...,movie standupcomedy NR,shannonhartman billburr
2485,70245034,movie,Bill Burr: You People Are All the Same,jaykaras,billburr,unitedstates,"June 30, 2018",2012,NR,69 min,standupcomedy,funnyman bill burr takes the stage to uncork a...,movie standupcomedy NR,jaykaras billburr
3237,80133549,movie,Bill Burr: Walk Your Way Out,jaykaras,billburr,unitedstates,"January 31, 2017",2017,TVMA,78 min,standupcomedy,no-nonsense comic bill burr takes the stage in...,movie standupcomedy TVMA,jaykaras billburr
3530,60031236,movie,Kill Bill: Vol. 1,quentintarantino,umathurman lucyliu vivicaafox darylhannah davi...,unitedstates japan,"January 1, 2020",2003,R,111 min,actionadventure,"an assassin is shot by her ruthless employer, ...",movie actionadventure R,quentintarantino umathurman lucyliu vivicaafox...
3531,60032563,movie,Kill Bill: Vol. 2,quentintarantino,umathurman davidcarradine michaelmadsen darylh...,unitedstates,"January 1, 2020",2004,R,137 min,actionadventure,the bride has three left on her rampage list: ...,movie actionadventure R,quentintarantino umathurman davidcarradine mic...
3759,80119555,movie,Blinky Bill: The Movie,deanetaylor,ryankwanten rufussewell tonicollette robinmcle...,australia unitedstates,"February 8, 2017",2015,PG,91 min,childrenfamilymovies,happy koala blinky bill and some spunky pals t...,movie childrenfamilymovies PG,deanetaylor ryankwanten rufussewell tonicollet...
3794,81002591,movie,Billy Graham: An Extraordinary Journey,vondaharrell danielcamenisch,billygraham jeffhoyt tombrokaw gretavansustere...,unitedstates,"February 28, 2019",2018,TVPG,66 min,documentaries faithspirituality,this documentary recounts the life story of ev...,movie documentaries faithspirituality TVPG,vondaharrell danielcamenisch billygraham jeffh...
4009,60003378,movie,Billy Elliot,stephendaldry,jamiebell garylewis stuartwells billyfane coli...,unitedkingdom france,"February 1, 2019",2000,R,110 min,dramas independentmovies internationalmovies,when a boy trades boxing school for ballet les...,movie dramas independentmovies internationalmo...,stephendaldry jamiebell garylewis stuartwells ...


In [35]:
np.save('app/static/npy/cosine_sim.npy',cosine_distance) #save cosine for server

In [36]:
#create title file for autocompleting in frontend (server)
title_array = []
for index, i in raw_data.iterrows():
    title = i['title']
    title = title.replace(" &", "")
    release_year = i['release_year']
    data = title + " (" + str(release_year)+ ")"
    title_array.append(data)

data = pd.DataFrame(title_array)

data.to_csv('app/static/posters/titles.csv', index= False, header=False)